IMDB データセットをダウンロードする

In [ ]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

訓練用データの中身をのぞいてみる

0がnegativeで、1がpositive

In [ ]:
train_data[0]
train_labels[0]

整数のインデックスになっているデータをテキストデータに変換

In [ ]:
word_index = imdb.get_word_index()

ダウンロードした辞書によって、整数のインデックスのリストで表されているデータを文字列にデコードします。
3を指定しているのは、0,1,2が学習には不要な文字のインデックスになっているためです。１番目のデータのテキストを表示してみます。

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

decoded_review

データは、整数のインデックスになっており、このままでは学習の入力に使えないので、One-hotベクトルに変換します。ここでは、One-hotベクトルの次元は10000とし、出現した単語に1を割り当てます。このOne-hotベクトルへの変換では、単語の出現頻度や並び順といった情報は落ちています。

In [ ]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

x_train[0]

またラベルも整数ではなく、浮動小数に変換します。

In [ ]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

y_train

次に学習させるネットワークを定義します。layers.Denseを使用して、全結合層のみのネットワークとなります。ネットワークとしては、3層の浅いニューラルネットで、1,2番目の隠れ層のユニットは16、活性化関数はreluを使用します。また、input_shapeにOne-hotベクトルの次元を指定します。２クラス分類なので、最後の層の出力はsigmoid層で、どちらのクラスであるかの確率を出力します。

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

モデル情報は以下のように出力できます。

In [ ]:
model.summary()

学習の際のoptimizer、損失関数、metricsを指定します。

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

学習データをtrain用と、validation用に分けます。今回はデータの前から10000個をvalidation用にしました。ここの分け方は、positiveとnegativeの数に不均衡が起きないようにランダムに分けたほうが精度がよくなるかも知れません。

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

学習は以下のようにmodel.fitを実行して行います。epochsは20、バッチサイズは512に今回は設定しました。

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))
history_dict = history.history

model.fitはhistory objectを返します。history objectはメンバのhistoryを持っていて学習の際の情報を辞書として持っています。

In [ ]:
history_dict.keys()

lossの変化をグラフに表示してみます。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc = history_dict['accuracy']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

精度についても以下のコードを実行して確認してみます。

In [ ]:
val_acc_values = history_dict['val_accuracy']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

testデータに対する精度を確認

In [ ]:
results = model.evaluate(x_test, y_test)
print(results)